In [1]:
import kagglehub
import pandas as pd
import os
from scipy.stats import skew, kurtosis

/home/stargazer/dev/AirBnB_Information_Visualisation_G6_2025/data_preparation/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 1. Start: download dataset

In [2]:
print("⚡︎ Downloading dataset...")
path = kagglehub.dataset_download("kritikseth/us-airbnb-open-data")
print(f"✓ Dataset downloaded to: {path}")

⚡︎ Downloading dataset...
✓ Dataset downloaded to: /home/stargazer/.cache/kagglehub/datasets/kritikseth/us-airbnb-open-data/versions/2


#### 2. Start: listing CSV files

In [3]:
print("⚡︎ Scanning dataset folder for CSV files...")
csv_files = [f for f in os.listdir(path) if f.lower().endswith(".csv")]
print(f"✓ CSV files found: {csv_files}")

dataframes = {}
for csv in csv_files:
    print("\n")
    print(f"⚡︎ Loading {csv} ...")
    csv_path = os.path.join(path, csv)
    df = pd.read_csv(csv_path, low_memory=False)
    dataframes[csv] = df
    print(f"✓ {csv} loaded → shape: {df.shape}")

⚡︎ Scanning dataset folder for CSV files...
✓ CSV files found: ['AB_US_2020.csv', 'AB_US_2023.csv']


⚡︎ Loading AB_US_2020.csv ...
✓ AB_US_2020.csv loaded → shape: (226030, 17)


⚡︎ Loading AB_US_2023.csv ...
✓ AB_US_2023.csv loaded → shape: (232147, 18)


#### 3. Start: check for 2020 and 2023

In [4]:
print("⚡︎ Checking for specific datasets (2020 & 2023)...")
df_2020 = dataframes.get("AB_US_2020.csv")
df_2023 = dataframes.get("AB_US_2023.csv")
print(f"✓ Found 2020 dataset: {df_2020 is not None}")
print(f"✓ Found 2023 dataset: {df_2023 is not None}")

⚡︎ Checking for specific datasets (2020 & 2023)...
✓ Found 2020 dataset: True
✓ Found 2023 dataset: True


#### 4. Clean up: drop unwanted columns if they exist

In [5]:
print("⚡︎ Cleaning up datasets...")
print(f". Initial 2020 dataset shape: {df_2020.shape}")
print(f". Initial 2023 dataset shape: {df_2023.shape}")

to_drop = ["neighbourhood_group", "number_of_reviews_ltm"]

for col in to_drop:
    if col in df_2020.columns:
        df_2020 = df_2020.drop(columns=col)
    if col in df_2023.columns:
        df_2023 = df_2023.drop(columns=col) 



# Make sure the columns match
assert list(df_2020.columns) == list(df_2023.columns), (
    "Columns are not the same after cleanup!"
)

print(f"✓ 2020 dataset shape after cleanup: {df_2020.shape}")
print(f"✓ 2023 dataset shape after cleanup: {df_2023.shape}")

⚡︎ Cleaning up datasets...
. Initial 2020 dataset shape: (226030, 17)
. Initial 2023 dataset shape: (232147, 18)
✓ 2020 dataset shape after cleanup: (226030, 16)
✓ 2023 dataset shape after cleanup: (232147, 16)


#### 5. Add 'year' column and merge

In [6]:
df_2020["year"] = "2020"
df_2023["year"] = "2023"

df = pd.concat([df_2020, df_2023], ignore_index=True)

print(f"✓ Merged dataset shape: {df.shape}")
print(f"✓ Columns: {df.columns.tolist()}")

✓ Merged dataset shape: (458177, 17)
✓ Columns: ['id', 'name', 'host_id', 'host_name', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'city', 'year']


#### 6. Map each city to its US state

In [7]:
print("⚡︎ Correcting city typos...")
df["city"] = df["city"].replace({
    "San Clara Country": "Santa Clara County"
})

city_to_state = {
    "New York City": "NY",
    "Los Angeles": "CA",
    "Broward County": "FL",
    "San Diego": "CA",
    "Austin": "TX",
    "Hawaii": "HI",
    "Clark County": "NV",
    "Nashville": "TN",
    "Chicago": "IL",
    "San Francisco": "CA",
    "Washington D.C.": "DC",
    "New Orleans": "LA",
    "Seattle": "WA",
    "Twin Cities MSA": "MN",
    "Denver": "CO",
    "Portland": "OR",
    "Rhode Island": "RI",
    "Boston": "MA",
    "Santa Clara County": "CA",
    "San Mateo County": "CA",
    "Oakland": "CA",
    "Asheville": "NC",
    "Jersey City": "NJ",
    "Columbus": "OH",
    "Santa Cruz County": "CA",
    "Cambridge": "MA",
    "Salem": "MA",
    "Pacific Grove": "CA"
}

print("⚡︎ Mapping cities to states...")
df["state"] = df["city"].map(city_to_state)
print(f"✓ 'state' column added with {df['state'].nunique()} unique values.")

# Check if any city didn't get mapped
missing = df[df["state"].isna()]["city"].unique()
if len(missing):
    print(f"! Some cities are missing state mappings: {missing}")
    raise ValueError("Some cities are missing state mappings!" + str(missing))
pd.set_option('display.float_format', '{:.2f}'.format)

⚡︎ Correcting city typos...
⚡︎ Mapping cities to states...
✓ 'state' column added with 19 unique values.


#### 7. Remove semantic duplicates (same host/listing details but different id)

In [8]:
print("⚡︎ Removing semantic duplicates (same host/listing details but different id)...")
before = df.shape
df = df.drop_duplicates(
    subset=["host_id", "name", "latitude", "longitude", "room_type", "price",
            "minimum_nights", "availability_365", "city", "year"],
    keep="first"
)
print(f"✓ Removed {before[0] - df.shape[0]} duplicates. New shape: {df.shape}")

⚡︎ Removing semantic duplicates (same host/listing details but different id)...
✓ Removed 971 duplicates. New shape: (457206, 18)


#### 8. Remove duplicate of ids

In [9]:
print(f". Initial dataset shape before collapsing: {df.shape}")
before = df.shape[0]
df = df.drop_duplicates(subset=['id', 'year']).copy() # id <-> year
after = df.shape[0]
print(f"✓ Removed {before - after} duplicates based on (id, year).")

. Initial dataset shape before collapsing: (457206, 18)
✓ Removed 2 duplicates based on (id, year).


#### 9. Filter minimum_nights to a reasonable range

In [10]:
print("⚡︎ Filtering 'minimum_nights' to a reasonable range...")
print(f". Initial dataset shape: {df.shape}")

# Identify invalid entries before filtering
invalid_mask = (df["minimum_nights"] <= 0) | (df["minimum_nights"] >= 2000)
removed_count = invalid_mask.sum()

if removed_count > 0:
    removed_min = df.loc[invalid_mask, "minimum_nights"].min()
    removed_max = df.loc[invalid_mask, "minimum_nights"].max()
    print(f"!  {removed_count:,} rows removed — values outside [1, 2000].")
    print(f"   -> Outlier range detected: min={removed_min}, max={removed_max}")
else:
    print("✓ No invalid 'minimum_nights' values found.")

# Apply filtering
df = df[~invalid_mask].copy()

print(f"✓ Dataset shape after filtering: {df.shape}")


⚡︎ Filtering 'minimum_nights' to a reasonable range...
. Initial dataset shape: (457204, 18)
!  1 rows removed — values outside [1, 2000].
   -> Outlier range detected: min=100000000, max=100000000
✓ Dataset shape after filtering: (457203, 18)


#### 10. Recompute host listing count

In [11]:
print("⚡︎ Recomputing listings per host/year...")
df['calculated_host_listings_count'] = (
    df.groupby(['host_id', 'year'])['id'].transform('nunique')
)
print("✓ Recomputed host listing counts successfully.")

⚡︎ Recomputing listings per host/year...
✓ Recomputed host listing counts successfully.


#### 11. Dataset validation

In [12]:
print("\n" + "=" * 80)
print("📋  FINAL DATASET REPORT")
print("=" * 80)

invalid_count = 0

# --- 1. BASIC STRUCTURE ---
print("\n🔹 [1] Dataset Structure Overview")
print("-" * 80)
print(f"• Total rows: {df.shape[0]:,}")
print(f"• Total columns: {df.shape[1]}")
print(f"• Columns available: {', '.join(df.columns)}")

# --- 2. MISSING VALUES ---
print("\n🔹 [2] Missing Value Audit")
print("-" * 80)
missing = df.isna().sum()
if missing.any():
    invalid_count += 1
    print("⚠️  Missing values detected in the following columns:")
    print(missing[missing > 0])
else:
    print("✓ No missing values found in any column.")

# --- 3. STATISTICAL DISTRIBUTION (Threshold-based Descriptive Report) ---
print("\n🔹 [3] Statistical Distribution Overview")
print("-" * 80)

thresholds = {
    "price": {"low": 20, "high": 10000},
    "minimum_nights": {"low": 1, "high": 100},
    "availability_365": {"low": 0, "high": 365},
    "reviews_per_month": {"low": 0, "high": 30},
}

for col, thr in thresholds.items():
    if col not in df.columns:
        print(f"⚠️  Column '{col}' not found in dataset.")
        invalid_count += 1
        continue

    series = df[col].dropna()
    mean, std = series.mean(), series.std()
    s, k = skew(series), kurtosis(series)
    min_val, max_val = series.min(), series.max()
    below = series[series < thr["low"]]
    above = series[series > thr["high"]]
    total_extreme = len(below) + len(above)
    ratio = 100 * total_extreme / len(series)

    print(f"\n📊  {col.upper()}  —  {len(series):,} observations")
    print(f"   Mean: {mean:.2f}   |   Std: {std:.2f}")
    print(f"   Min: {min_val:.2f}   |   Max: {max_val:.2f}")
    print(f"   Skew: {s:.2f}   |   Kurtosis: {k:.2f}")
    print(f"   Thresholds of interest → below {thr['low']} or above {thr['high']}")

    if total_extreme > 0:
        print(f"   • {total_extreme:,} values ({ratio:.2f}%) fall beyond thresholds:")
        if len(below):
            print(f"     - {len(below):,} below {thr['low']}  (min={below.min():.2f}, max={below.max():.2f})")
        if len(above):
            print(f"     - {len(above):,} above {thr['high']} (min={above.min():.2f}, max={above.max():.2f})")
    else:
        print("   • All observations fall within defined thresholds.")

    # Shape commentary
    shape = "approximately symmetric"
    if s > 0.5:
        shape = "right-skewed (long tail on high values)"
    elif s < -0.5:
        shape = "left-skewed (long tail on low values)"

    if k > 3:
        tail = "leptokurtic — sharply peaked with heavy tails"
    elif k < 3:
        tail = "platykurtic — flatter, lighter tails"
    else:
        tail = "mesokurtic — similar to a normal curve"

    print(f"   • Distribution is {shape} and {tail}.")
    print("-" * 80)

# --- 4. LOGICAL UNIQUENESS ---
print("\n🔹 [4] Logical Consistency Checks")
print("-" * 80)
dup_id_year = df.duplicated(subset=["id", "year"]).sum()
if dup_id_year:
    invalid_count += 1
    print(f"⚠️  {dup_id_year} duplicate (id, year) pairs found.")
else:
    print("✓ Each (id, year) pair is unique — no duplicate listings per year.")

# --- 5. HOST–LISTING CONSISTENCY ---
print("\n🔹 [5] Host–Listing–Year Consistency")
print("-" * 80)
for year, group in df.groupby("year"):
    total_listings = group["id"].nunique()
    total_from_hosts = group.groupby("host_id")["calculated_host_listings_count"].first().sum()
    if total_listings == total_from_hosts:
        print(f"✓ {year}: Consistent — {total_listings:,} listings across {group['host_id'].nunique():,} hosts.")
    else:
        print(f"⚠️  {year}: Mismatch — listings={total_listings}, summed host counts={total_from_hosts}")
        invalid_count += 1

# --- 6. CATEGORICAL SANITY ---
print("\n🔹 [6] Categorical Column Audit")
print("-" * 80)
categorical_cols = ["city", "state", "room_type", "year", "neighbourhood", "host_name"]
for col in categorical_cols:
    if col in df.columns:
        unique_vals = df[col].nunique()
        print(f"✓ '{col}' contains {unique_vals} unique values.")
    else:
        print(f"⚠️  Column '{col}' missing from dataset.")
        invalid_count += 1

# --- 7. GEOGRAPHIC VALIDATION ---
print("\n🔹 [7] Geographic Coordinate Validation")
print("-" * 80)
if "latitude" in df.columns and "longitude" in df.columns:
    invalid_lat = (~df["latitude"].between(-90, 90)).sum()
    invalid_lon = (~df["longitude"].between(-180, 180)).sum()
    if invalid_lat or invalid_lon:
        invalid_count += 1
        print(f"⚠️  {invalid_lat} invalid latitudes and {invalid_lon} invalid longitudes found.")
    else:
        print("✓ All coordinates fall within valid Earth ranges (±90°, ±180°).")
else:
    print("⚠️  Latitude/Longitude columns missing.")
    invalid_count += 1

# --- 8. FINAL SUMMARY ---
print("\n" + "=" * 80)
print("📋  VALIDATION SUMMARY")
print("=" * 80)
if invalid_count > 0:
    print(f"⚠️  Validation completed with {invalid_count} potential issues detected.")
    print("   Please review warnings above before export.")
else:
    print("✅  Validation passed — dataset is clean, consistent, and ready for export.")
print("=" * 80)


📋  FINAL DATASET REPORT

🔹 [1] Dataset Structure Overview
--------------------------------------------------------------------------------
• Total rows: 457,203
• Total columns: 18
• Columns available: id, name, host_id, host_name, neighbourhood, latitude, longitude, room_type, price, minimum_nights, number_of_reviews, last_review, reviews_per_month, calculated_host_listings_count, availability_365, city, year, state

🔹 [2] Missing Value Audit
--------------------------------------------------------------------------------
⚠️  Missing values detected in the following columns:
name                    44
host_name               46
last_review          96990
reviews_per_month    96990
dtype: int64

🔹 [3] Statistical Distribution Overview
--------------------------------------------------------------------------------

📊  PRICE  —  457,203 observations
   Mean: 238.94   |   Std: 822.68
   Min: 0.00   |   Max: 100000.00
   Skew: 58.59   |   Kurtosis: 5822.14
   Thresholds of interest → bel

#### 12. Translate ZIP codes to neighbourhood names


In [13]:
print("⚡︎ Translating ZIP codes to neighbourhood names...")

# Install pgeocode if needed: pip install pgeocode
import pgeocode

# Initialize US geocoder
nomi = pgeocode.Nominatim('us')

def translate_neighbourhood(value):
    """
    If value is numeric (ZIP code), translate to place name.
    Otherwise, keep as is.
    """
    if pd.isna(value):
        return value
    
    # Convert to string and check if it's numeric
    str_value = str(value).strip()
    
    # Check if it looks like a ZIP code (5 digits)
    if str_value.isdigit() and len(str_value) == 5:
        try:
            # Query the ZIP code
            result = nomi.query_postal_code(str_value)
            
            # Check if we got a valid result
            if not pd.isna(result.place_name):
                # Return the place name (city/town name)
                return result.place_name
            else:
                # If no result found, keep the ZIP code
                return str_value
        except Exception as e:
            # On any error, keep original value
            print(f"   Warning: Could not translate ZIP {str_value}: {e}")
            return str_value
    else:
        # Not a ZIP code, keep as is
        return str_value

# Apply translation
print("   Processing neighbourhood values...")
original_count = df['neighbourhood'].nunique()
df['neighbourhood'] = df['neighbourhood'].apply(translate_neighbourhood)
new_count = df['neighbourhood'].nunique()

print(f"✓ Neighbourhood translation completed.")
print(f"   Original unique values: {original_count:,}")
print(f"   After translation: {new_count:,}")


⚡︎ Translating ZIP codes to neighbourhood names...
   Processing neighbourhood values...
✓ Neighbourhood translation completed.
   Original unique values: 1,459
   After translation: 1,411


#### 13. Export to CSV

In [14]:
output_dir = "out"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "dataset.csv")

# Explicitly remove old file if it exists
if os.path.exists(output_path):
    os.remove(output_path)
    print(f". Old file removed: {output_path}")

print(f". Exporting cleaned dataset to: {output_path}")
df.to_csv(output_path, index=False)
print("✓ CSV export completed successfully.")

. Old file removed: out/dataset.csv
. Exporting cleaned dataset to: out/dataset.csv
✓ CSV export completed successfully.
